In [55]:
import requests
import json

endpoint = "http://robokop.renci.org:6434/query?limit=20&strict=false"

query = json.loads("""
{
  "message": {
    "query_graph": {
      "nodes": [
        {
          "id": "n0",
          "type": "gene",
          "curie": "NCBIGENE:558"
        },
        {
          "id": "n1",
          "type": "biological_process"
        },
        {
          "id": "n2",
          "type": "chemical_substance"
        },
        {
          "id": "n4",
          "type": "biological_process",
          "curie": "GO:0002376"
        }
      ],
      "edges": [
        {
          "id": "e0",
          "source_id": "n1",
          "target_id": "n0",
          "type": "has_participant"
        },
        {
          "id": "e1",
          "source_id": "n2",
          "target_id": "n1",
          "type": "causes"
        },
        {
          "id": "e2",
          "source_id": "n4",
          "target_id": "n0",
          "type": "has_participant"
        }
      ]
    }
  }
}
""")
queryGraph = query["message"]["query_graph"]
r = requests.post(endpoint, json=query)
result = r.json()
resultGraph = result["knowledge_graph"]

#print(json.dumps(resultGraph, indent=4))

In [56]:
from cyjupyter import Cytoscape

def reasonerGraphToCytoscape(graph):
    csGraph = {}
    nodes = []
    edges = []
    for node in graph["nodes"]:
        csNode = {}
        node_types = ""
        if isinstance(node["type"], str):
            node_types = node["type"]
        else:
            node_types = "\n".join(node["type"])
        csNode["data"] = {"id": node["id"], "label": node_types + "\n[" + node.get("curie", "") + "]", "curie": node.get("curie", ""), "type": node_types}
        nodes.append(csNode)
    for edge in graph["edges"]:
        csEdge = {
            "data": {
                "id": edge["id"],
                "source": edge["source_id"],
                "target": edge["target_id"],
                "label": edge["type"]
            }
        }
        edges.append(csEdge)
    csGraph["elements"] = {"nodes": nodes, "edges": edges}
    csGraph["style"] = [
        { "selector": 'node', "style": {
            'label': 'data(label)',
            'color': 'white',
            'background-color': '#60f', # #009 looks good too
            'shape': 'rectangle',
            'text-valign': 'center',
            'text-border-style': 'solid',
            'text-border-width': 5,
            'text-border-color': 'red',
            'width': '20em',
            'height': '5em',
            'text-wrap': 'wrap'
        } }, 
        {"selector": "edge", "style": {
            "curve-style": "unbundled-bezier",
            # "control-point-distances": [20, -20],
            # "control-point-weights": [0.250, 0.75],
            "control-point-distances": [-20, 20],
            "control-point-weights": [0.5],
            'content': 'data(label)',
            'line-color': '#808080',
            'target-arrow-color': '#808080',
            'target-arrow-shape': 'triangle',
            'target-arrow-fill': 'filled'}
        }
    ]
                       
    #print(json.dumps(csGraph, indent=4))
    return csGraph

def knowledgeGraphToCytoscape(graph):
    csGraph = {}
    nodes = []
    edges = []
    for node in graph["nodes"]:
        csNode = {}
        node_types = ""
        if isinstance(node["type"], str):
            node_types = node["type"]
        else:
            node_types = "\n".join(node["type"])
        csNode["data"] = {"id": node["id"], "label": (node["name"] or " ") + "\n[" + node["id"] + "]", "curie": node["id"], "type": node_types}
        nodes.append(csNode)
    for edge in graph["edges"]:
        csEdge = {
            "data": {
                "id": edge["id"],
                "source": edge["source_id"],
                "target": edge["target_id"],
                "label": edge["type"]
            }
        }
        edges.append(csEdge)
    csGraph["elements"] = {"nodes": nodes, "edges": edges}
    csGraph["style"] = [
        { "selector": 'node', "style": {
            'label': 'data(label)',
            'color': 'white',
            'background-color': '#60f', # #009 looks good too
            'shape': 'rectangle',
            'text-valign': 'center',
            'text-border-style': 'solid',
            'text-border-width': 5,
            'text-border-color': 'red',
            'width': '20em',
            'height': '5em',
            'text-wrap': 'wrap'
        } }, 
        {"selector": "edge", "style": {
            "curve-style": "unbundled-bezier",
            # "control-point-distances": [20, -20],
            # "control-point-weights": [0.250, 0.75],
            "control-point-distances": [-20, 20],
            "control-point-weights": [0.5],
            'content': 'data(label)',
            'line-color': '#808080',
            'target-arrow-color': '#808080',
            'target-arrow-shape': 'triangle',
            'target-arrow-fill': 'filled'}
        }
    ]
                       
    #print(json.dumps(csGraph, indent=4))
    return csGraph
queryData = reasonerGraphToCytoscape(queryGraph)
Cytoscape(data=queryData, visual_style=queryData["style"], layout={"name": "cose", "height": "700px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'gene\n[NCBIGENE:558]', 'curie': 'NCBIGE…

In [57]:
Cytoscape(
    data=knowledgeGraphToCytoscape(resultGraph),
    visual_style=queryData["style"],
    layout_name='cose',
    layout={
        "height": "700px"
    }
)

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'http://identifiers.org/mgi/MGI:3528396', 'label': 'Iqs…